In [1]:
import torch

In [4]:
nbeats = torch.load('/home/jupyter/imported/lightning_logs/version_3/checkpoints/epoch=19-step=599.ckpt')

In [95]:
import os
import gc
import numpy as np
import pandas as pd

data_path = '../input/g-research-crypto-forecasting/train.csv'
dtypes = {
    'timestamp': np.int64,
    'Asset_ID': np.int8,
    'Count': np.float32,
    'Open': np.float32,
    'High': np.float32,
    'Low': np.float32,
    'Close': np.float32,
    'Volume': np.float32,
    'VWAP': np.float32,
    'Target': np.float32
}
train = pd.read_csv(data_path, dtype=dtypes, usecols=list(dtypes.keys()))

asset_details_path = '../input/g-research-crypto-forecasting/asset_details.csv'
asset_details = pd.read_csv(asset_details_path)
assets_order = dict((t,i) for i,t in enumerate(asset_details.Asset_ID))

crypto_df = train.merge(asset_details, on='Asset_ID')
crypto_df['timestamp'] = pd.to_datetime(crypto_df['timestamp'], unit='s')

In [96]:
crypto_df = crypto_df.sort_values('timestamp', ascending=False).groupby('Asset_ID').head(5000)

In [97]:
# essentially verbatim from https://www.kaggle.com/alexfir/recreating-target/
def reconstruct_targets(data: pd.DataFrame, details: pd.DataFrame, price_column: str, use_log=False):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_Name)
    weights = np.array(list(details.Weight))
    
    timestamp_series = pd.Series([pd.Timestamp("2018-01-01 00:00")]).append(crypto_df['timestamp'], ignore_index=True)
    all_timestamps = np.sort(timestamp_series.unique())

    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        if use_log:
            targets[asset_names[i]] = np.log(price.shift(periods=-16)) - np.log(price.shift(periods=-1))
        else:
            targets[asset_names[i]] = (
                price.shift(periods=-16) /
                price.shift(periods=-1)
            ) - 1
            
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    m = targets['m']
    
    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)
    
    recon_targets = targets - (beta * m.values).T
    recon_targets.drop('m', axis=1, inplace=True)
    
    return recon_targets

In [98]:
def merge_recon_with_train(crypto_df, asset_details, price_column='Close', replace=0):
    recon_targets = reconstruct_targets(data=crypto_df, details=asset_details, price_column=price_column)
    recon_targets = pd.melt(recon_targets.reset_index(), id_vars='index')
    if replace:
        recon_targets = recon_targets.rename(columns={'index':'timestamp', 'variable':'Asset_Name', 'value':'Target'})
        recon_targets = recon_targets.sort_values(['timestamp']).groupby('Asset_Name').tail(replace)
        crypto_df = crypto_df.sort_values(['timestamp','Asset_Name']).set_index(['timestamp','Asset_Name'])
        recon_targets = recon_targets.sort_values(['timestamp','Asset_Name']).set_index(['timestamp','Asset_Name'])
        crypto_df.update(recon_targets)
        return crypto_df.reset_index(), recon_targets
    else:
        recon_targets = recon_targets.rename(columns={'index':'timestamp', 'variable':'Asset_Name', 'value':'recon_Target'})
        return crypto_df.merge(recon_targets, on=['Asset_Name', 'timestamp'])

In [99]:
df = merge_recon_with_train(crypto_df[['timestamp','Asset_Name','Asset_ID','Close','Target']], asset_details, replace = 0)

In [100]:
df

,timestamp,Asset_Name,Asset_ID,Close,Target,recon_Target
0,2021-09-21 00:00:00,Dogecoin,4,0.208294,NaN,NaN
1,2021-09-21 00:00:00,Binance Coin,0,363.757507,NaN,NaN
2,2021-09-21 00:00:00,Cardano,3,2.079268,NaN,NaN
3,2021-09-21 00:00:00,Litecoin,9,156.943863,NaN,NaN
4,2021-09-21 00:00:00,Ethereum Classic,7,49.068150,NaN,NaN
...,...,...,...,...,...,...
69995,2021-09-17 12:22:00,Maker,10,2816.417236,-0.002473,-0.002473
69996,2021-09-17 12:21:00,Maker,10,2818.099854,-0.000613,-0.000613
69997,2021-09-17 12:20:00,Maker,10,2819.486816,-0.001527,-0.001527
69998,2021-09-17 12:19:00,Maker,10,2817.721924,-0.002492,-0.002492


In [85]:
recon_targets

Target
timestamp  Asset_Name              
2021-09-21 Binance Coin         NaN
           Bitcoin              NaN
           Bitcoin Cash         NaN
           Cardano              NaN
           Dogecoin             NaN
           EOS.IO               NaN
           Ethereum             NaN
           Ethereum Classic     NaN
           IOTA                 NaN
           Litecoin             NaN
           Maker                NaN
           Monero               NaN
           Stellar              NaN
           TRON                 NaN